In [112]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.tools.visualization import plot_histogram

import random


In [113]:
ndim = 1

In [114]:
from qiskit.quantum_info import Pauli
from itertools import product

# create n-qubit pauli operator set using cartesian product
def create_pauli(ndim):
    pauli_set = []
    for i in product(['I', 'X', 'Y', 'Z'], repeat=ndim):
        pauli_set.append(Pauli(''.join(i)))
    
    assert not(ndim==1) or len(pauli_set) == 4
    assert not(ndim ==2) or len(pauli_set) == 16

    return pauli_set

# # create 2-qubit pauli operator set
# pauli_set = create_pauli(ndim)

In [115]:
import numpy as np
from qiskit.quantum_info import random_clifford
from qiskit.quantum_info import Statevector

def create_stabilizer_set(ndim):
    """Monte-Carlo approach, set of states from random clifford group.
    By definition, the stabilizer states are created by Clifford gates acting on |0\>^{otimes n}
    """
    N_cliff = max(2,ndim)**8 # defines the number of random Clifford circuits to generate
    # NOTE increase until len(stabilizer_set) converges

    stabilizer_set = []
    for i in range(N_cliff):
        # run random clifford 
        qc = QuantumCircuit(ndim)
        qc.append(random_clifford(ndim), range(ndim))
        # circuit to state
        state = Statevector.from_instruction(qc)
        # append to set
        stabilizer_set.append(state)
        # print(state)
    
    # drop duplicates (up to a global phase)
    # use Statevector.equiv to check if two states are equivalent
    for i in range(len(stabilizer_set)):
        if stabilizer_set[i] is None:
            continue
        for j in range(i+1, len(stabilizer_set)):
            if stabilizer_set[j] is None:
                continue
            if stabilizer_set[i].equiv(stabilizer_set[j]):
                stabilizer_set[j] = None
    stabilizer_set = [x for x in stabilizer_set if x is not None]
    
    assert not(ndim==1) or len(stabilizer_set) == 6
    # for more than ndim, idk how many stabilizer states there are
    return stabilizer_set

The optimization can be written in terms of a linear system as $$\mathcal{R}(\rho) = \min || x ||_1 \text{ subject to } Ax = b$$, where $||x||_1 = \sum_i |x_i|$, $b_i = \mathrm{Tr}{(P_i \rho)}$, and $A_{j,i} = \mathrm{Tr}{(P_j \sigma_i)}$ where $P_j$ is the $j$-th Pauli operator.

In [123]:
# Reference: https://arxiv.org/abs/1609.07488

def construct_A(ndim, pauli_set=None):
    """Constructs the A matrix for the stabilizer states
    """
    if pauli_set is None:
        pauli_set = create_pauli(ndim)
    stabilizer_set = create_stabilizer_set(ndim)
    A = np.zeros((len(pauli_set), len(stabilizer_set)), dtype=complex)
    for i, state in enumerate(stabilizer_set):
        for j, pauli in enumerate(pauli_set):
            A[j,i] = state.expectation_value(pauli)
    return A

def construct_B(statevector, pauli_set=None):
    """Constructs the B matrix for the statevector
    """
    if pauli_set is None:
        pauli_set = create_pauli(ndim)
    B = np.zeros((len(pauli_set),), dtype=complex)
    for j, pauli in enumerate(pauli_set):
        B[j] = statevector.expectation_value(pauli)
    return B

# construct_A(1)
# construct_B(Statevector.from_label('0'))

In [135]:
import cvxpy as cp
# NOTE use cvxpy because nonunique solution
# we could solve Ax = b using numpy.linalg.lstsq
# x, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)

def robustness_of_magic(statevector) -> float:
    """Compute the robustness of a statevector"""
    # first determine size of problem
    ndim = int(np.log2(len(statevector)))
    if ndim > 2:
        raise ValueError("ndim too big, current implementation too slow")
    pauli_set = create_pauli(ndim)

    # second construct the A matrix and b vector
    A = construct_A(ndim, pauli_set)
    b = construct_B(statevector, pauli_set)

    # now solve the optimization problem 
    # min ||x||_1 s.t. Ax = b
    x = cp.Variable(len(A[0]))
    objective = cp.Minimize(cp.norm(x,1))
    constraints = [A @ x == b]
    prob = cp.Problem(objective, constraints)
    prob.solve()
    return prob.value

# now we can compute the robustness of the magic state
H = Statevector([1,np.exp(1j * np.pi/4)])/np.sqrt(2)
# H.draw('bloch')

# verify correctness of 1-qubit case
rH = robustness_of_magic(H)
print(rH)
assert np.isclose(rH, np.sqrt(2))

# verify correctness of 2-qubit case
H2 = H.tensor(H)
rH2 = robustness_of_magic(H2)
print(rH2)
assert np.isclose(rH2, (1 + 3*np.sqrt(2))/3)

# verify correctness of 3-qubit case
# scales so poorly :(
# H3 = H2.tensor(H)
# rH3 = robustness_of_magic(H3)
# print(rH3)
# assert np.isclose(rH3, (1 + 4*np.sqrt(2))/3)

1.4142135626928556
1.747546895825713


In [226]:
def robustness_to_gadgets(r, tolerance=0.1):
    thresholds = [1.4142, 1.7476, 2.2190, 2.8627, 3.68705]
    for i, threshold in enumerate(thresholds):
        if r <= threshold + 1e-6:
            return i 

In [227]:
# what is the robustness of CX?
def gate_gadgets(qc):
    # convert qc to statevector

    # by definition, apply U to |+>^{otimes n} to  get |U>
    # preprend Hadamard gates to qc
    h_prep = QuantumCircuit(qc.num_qubits)
    for i in range(qc.num_qubits):
        h_prep.h(i)
    qc = h_prep.compose(qc)

    statevector = Statevector.from_instruction(qc)
    # compute robustness
    r = robustness_of_magic(statevector)
    print("Robustness of Magic:", r)
    g = robustness_to_gadgets(r)
    print("Minimal # of gadgets:", g)
    return g

In [228]:
from qiskit.circuit.library import *
qc = QuantumCircuit(2)
# qc.append(CSGate(), [0,1])
qc.append(CXGate(), [0,1])
g = gate_gadgets(qc)

Robustness of Magic: 1.000000000007496
Minimal # of gadgets: 0


In [234]:
# next we can test how many gadgets would be used in qiskit implementations of gates
# gadget decompositions are known for t, tdg, ccx, u1
extended_stabilizer_simulator = AerSimulator(method='extended_stabilizer')
from qiskit.quantum_info import random_unitary
# transpile to CX and T
qc = QuantumCircuit(2)
qc.append(random_unitary(4), [0,1])
transp_qc = transpile(qc, extended_stabilizer_simulator)
g = gate_gadgets(qc)

Robustness of Magic: 2.1481448980727103
Minimal # of gadgets: 2


In [235]:
transp_qc.draw()

global phase: 0.87352
     ┌────────────┐┌────┐ ┌────────────┐┌────┐┌─────────────┐     »
q_0: ┤ P(-1.6619) ├┤ √X ├─┤ P(-1.3302) ├┤ √X ├┤ P(-0.32551) ├──■──»
     ├────────────┤├────┤┌┴────────────┤├────┤└┬───────────┬┘┌─┴─┐»
q_1: ┤ P(-1.8821) ├┤ √X ├┤ P(-0.65957) ├┤ √X ├─┤ P(1.5642) ├─┤ X ├»
     └────────────┘└────┘└─────────────┘└────┘ └───────────┘ └───┘»
«       ┌─────────┐  ┌────┐┌────────────┐┌────┐  ┌────────┐       ┌─────────┐ »
«q_0: ──┤ P(-π/2) ├──┤ √X ├┤ P(-2.1119) ├┤ √X ├──┤ P(π/2) ├───■───┤ P(-π/2) ├─»
«     ┌─┴─────────┴─┐├────┤├────────────┤├────┤┌─┴────────┴┐┌─┴─┐┌┴─────────┴┐»
«q_1: ┤ P(-0.98651) ├┤ √X ├┤ P(-2.3213) ├┤ √X ├┤ P(2.7179) ├┤ X ├┤ P(2.6039) ├»
«     └─────────────┘└────┘└────────────┘└────┘└───────────┘└───┘└───────────┘»
«     ┌────┐┌────────────┐┌────┐                   ┌────────────┐┌────┐»
«q_0: ┤ √X ├┤ P(-3.0607) ├┤ √X ├────────────────■──┤ P(-2.6675) ├┤ √X ├»
«     ├────┤├────────────┤├────┤┌────────────┐┌─┴─┐├────────────┤├────┤»
«q_1: ┤ √X ├┤ P(-1.0508) ├┤ √X ├┤ P(0.69476) ├┤ X ├┤ P(-0.5608) ├┤ √X ├»
«     └────┘└────────────┘└────┘└────────────┘└───┘└────────────┘└────┘»
«     ┌────────────┐┌────┐ ┌───────────┐ 
«q_0: ┤ P(-2.2493) ├┤ √X ├─┤ P(2.7528) ├─
«     ├────────────┤├────┤┌┴───────────┴┐
«q_1: ┤ P(-1.0266) ├┤ √X ├┤ P(-0.16643) ├
«     └────────────┘└────┘└─────────────┘